In [1]:
import tensorflow as tf
import cv2
import json
import numpy as np
from matplotlib import pyplot as plt

In [2]:
def load_image(x):
    byte_img = tf.io.read_file(x)
    img = tf.io.decode_jpeg(byte_img)
    return img

In [3]:
train_images = tf.data.Dataset.list_files('aug_data\\train\\images\\*.jpg' , shuffle=False)
train_images = train_images.map(load_image)
train_images = train_images.map(lambda x: tf.image.resize(x , (120,120)))
train_images = train_images.map(lambda x: x/255)

In [4]:
test_images = tf.data.Dataset.list_files('aug_data\\test\\images\\*.jpg' , shuffle=False)
test_images = test_images.map(load_image)
test_images = test_images.map(lambda x: tf.image.resize(x , (120,120)))
test_images = test_images.map(lambda x: x/255)

In [5]:
val_images = tf.data.Dataset.list_files('aug_data\\val\\images\\*.jpg' , shuffle=False)
val_images = val_images.map(load_image)
val_images = val_images.map(lambda x: tf.image.resize(x , (120,120)))
val_images = val_images.map(lambda x: x/255)

##### Prepare label

In [6]:
def load_labels(label_path):
    with open(label_path.numpy() , 'r' , encoding='utf-8') as f:
        label = json.load(f)
        
    return [label['class']] , label['bbox']

In [7]:
train_labels = tf.data.Dataset.list_files('aug_data\\train\\labels\\*.json' , shuffle=False)
train_labels = train_labels.map(lambda x:tf.py_function(load_labels , [x] , [tf.uint8 , tf.float16]))

In [8]:
test_labels = tf.data.Dataset.list_files('aug_data\\test\\labels\\*.json' , shuffle=False)
test_labels = test_labels.map(lambda x:tf.py_function(load_labels , [x] , [tf.uint8 , tf.float16]))

In [9]:
val_labels = tf.data.Dataset.list_files('aug_data\\val\\labels\\*.json' , shuffle=False)
val_labels = val_labels.map(lambda x:tf.py_function(load_labels , [x] , [tf.uint8 , tf.float16]))

#### Create final dataset


In [10]:
train = tf.data.Dataset.zip((train_images , train_labels))
train = train.shuffle(5000)
train = train.batch(8)
train = train.prefetch(4)

In [11]:
test = tf.data.Dataset.zip((train_images , train_labels))
test = test.shuffle(5000)
test = test.batch(8)
test = test.prefetch(4)

In [12]:
val = tf.data.Dataset.zip((train_images , train_labels))
val = val.shuffle(5000)
val = val.batch(8)
val = val.prefetch(4)

# Strat with model training

In [13]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Dense, GlobalMaxPooling2D
from tensorflow.keras.applications import VGG16

In [14]:
vgg = VGG16(include_top=False)

In [15]:
def build_model():
    input_layer = Input(shape=(120 , 120 , 3))
    
    vgg = VGG16(include_top=False)(input_layer)
    
    # Classification model
    f1 = GlobalMaxPooling2D()(vgg)
    class1 = Dense(2048 , activation='relu')(f1)
    class2 = Dense(1 , activation='sigmoid')(class1)
    
    # Bounding box model
    f2 = GlobalMaxPooling2D()(vgg)
    regress1 = Dense(2048 , activation='relu')(f2)
    regress2 = Dense(4 , activation='sigmoid')(regress1)
    
    facetracker = Model(inputs=input_layer , outputs=[class2 , regress2])
    return facetracker

In [16]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, None, None, 128)   0     

In [17]:
facetracker = build_model()

In [18]:
facetracker.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 120, 120, 3  0           []                               
                                )]                                                                
                                                                                                  
 vgg16 (Functional)             (None, None, None,   14714688    ['input_2[0][0]']                
                                512)                                                              
                                                                                                  
 global_max_pooling2d (GlobalMa  (None, 512)         0           ['vgg16[0][0]']                  
 xPooling2D)                                                                                  

In [19]:
# Since we've a classification model and a regression model , we need seperate loss function for both.
batches_per_epoch = len(train)
lr_decay = (1./0.75-1)/batches_per_epoch

In [20]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001 , decay=lr_decay)

In [21]:
def localization_loss(y_true, yhat):            
    delta_coord = tf.reduce_sum(tf.square(y_true[:,:2] - yhat[:,:2]))
                  
    h_true = y_true[:,3] - y_true[:,1] 
    w_true = y_true[:,2] - y_true[:,0] 

    h_pred = yhat[:,3] - yhat[:,1] 
    w_pred = yhat[:,2] - yhat[:,0] 
    
    delta_size = tf.reduce_sum(tf.square(w_true - w_pred) + tf.square(h_true-h_pred))
    
    return delta_coord + delta_size

In [22]:
classloss = tf.keras.losses.BinaryCrossentropy()
regressloss = localization_loss

In [23]:
class FaceTracker(Model): 
    def __init__(self, eyetracker,  **kwargs): 
        super().__init__(**kwargs)
        self.model = eyetracker

    def compile(self, opt, classloss, localizationloss, **kwargs):
        super().compile(**kwargs)
        self.closs = classloss
        self.lloss = localizationloss
        self.opt = opt
    
    def train_step(self, batch, **kwargs): 
        
        X, y = batch
        
        with tf.GradientTape() as tape: 
            classes, coords = self.model(X, training=True)
            
            batch_classloss = self.closs(y[0], classes)
            batch_localizationloss = self.lloss(tf.cast(y[1], tf.float32), coords)
            
            total_loss = batch_localizationloss+0.5*batch_classloss
            
            grad = tape.gradient(total_loss, self.model.trainable_variables)
        
        opt.apply_gradients(zip(grad, self.model.trainable_variables))
        
        return {"total_loss":total_loss, "class_loss":batch_classloss, "regress_loss":batch_localizationloss}
    
    def test_step(self, batch, **kwargs): 
        X, y = batch
        
        classes, coords = self.model(X, training=False)
        
        batch_classloss = self.closs(y[0], classes)
        batch_localizationloss = self.lloss(tf.cast(y[1], tf.float32), coords)
        total_loss = batch_localizationloss+0.5*batch_classloss
        
        return {"total_loss":total_loss, "class_loss":batch_classloss, "regress_loss":batch_localizationloss}
        
    def call(self, X, **kwargs): 
        return self.model(X, **kwargs)

In [24]:
model = FaceTracker(facetracker)

In [25]:
model.compile(opt , classloss , regressloss)

In [26]:
logdir = 'logs'

In [27]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
hist = model.fit(train , epochs=40 , validation_data=val , callbacks=[tensorboard_callback])

Epoch 1/5
 91/180 [==============>...............] - ETA: 10:39 - total_loss: 4.7546 - class_loss: 0.0083 - regress_loss: 4.7504

In [ ]:
hist.history

In [ ]:
fig, ax = plt.subplots(ncols=3, figsize=(20,5))

ax[0].plot(hist.history['total_loss'], color='teal', label='loss')
ax[0].plot(hist.history['val_total_loss'], color='orange', label='val loss')
ax[0].title.set_text('Loss')
ax[0].legend()

ax[1].plot(hist.history['class_loss'], color='teal', label='class loss')
ax[1].plot(hist.history['val_class_loss'], color='orange', label='val class loss')
ax[1].title.set_text('Classification Loss')
ax[1].legend()

ax[2].plot(hist.history['regress_loss'], color='teal', label='regress loss')
ax[2].plot(hist.history['val_regress_loss'], color='orange', label='val regress loss')
ax[2].title.set_text('Regression Loss')
ax[2].legend()

plt.show()

##### Testing data

In [ ]:
test_data = test.as_numpy_iterator()
test_sample = test_data.next()
yhat = facetracker.predict(test_sample[0])

In [ ]:
fig , ax = plt.subplots(ncols=4 , figsize=(20,20))
for idx in range(4):
  sample_image = test_sample[0][idx]
  sample_coords = yhat[1][idx]

  if yhat[0][idx]>0.5:
        cv2.rectangle(sample_image, 
                      tuple(np.multiply(sample_coords[:2], [120,120]).astype(int)),
                      tuple(np.multiply(sample_coords[2:], [120,120]).astype(int)), 
                            (255,0,0), 2)
  ax[idx].imshow(sample_image)

##### Save model

In [ ]:
from tensorflow.keras.models import load_model
facetracker.save('facetracker.h5')